In [1]:
import os
import sys
import logging

import pandas as pd

# import seaborn as sns
# sns.set_theme()
# sns.set_color_codes()
import matplotlib.pyplot as plt
import numpy as np

import plotly.express as px
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots
import plotly.graph_objects as go


# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
sys.path.append("../components/paddock/")

%load_ext autoreload
%autoreload 2

from telemetry.influx import Influx
from telemetry.analyzer import Analyzer

from IPython.display import Image

pd.set_option("display.max_columns", None)

influx = Influx()
influx.laps_from_file("iracing.csv")
analyzer = Analyzer()

In [2]:
# (game, session, track, car) = "iRacing,1670222575,pocono 2016,Dallara IR18".split(",")
# (game, session, track, car) = "Automobilista 2,1671635969,Cascavel4:Cascavel2,Mercedes-Benz Actros".split(",")
# (game, session, track, car) = "iRacing,1670287161,spielberg gp,Porsche 911 GT3.R".split(",")
# (game, session, track, car) = "iRacing,1670238139,spa up,Mercedes W12".split(",")
# (game, session, track, car) = "iRacing,1671643370,charlotte 2018 2019 rallycrosslong,Volkswagen Beetle GRC Lite".split(",")
# (game, session, track, car) = "iRacing,1670846922,nurburgring nordschleife,Porsche 911 GT3 Cup (992)".split(",")
# (game, session, track, car) = "Automobilista 2,1670846922,Nurburgring_2020:Nordschleife_2020_24hr,Porsche Cayman GT4 Clubsport MR".split(",")
# (game, session, track, car) = "Assetto Corsa (64 bit),1670215352,simtraxx_zeran:rally1,porsche_550_1500_rs_spyder_s1".split(",")

# (game, session, track, car) = "iRacing,1670351687,fuji gp,Ferrari 488 GT3 Evo 2020".split(",")
# (game, session, track, car) = "iRacing,1673613841,sebring international,Ferrari 488 GT3 Evo 2020".split(",")
# (game, session, track, car) = "iRacing,1673537226,oulton international,Mazda MX-5 Cup".split(",")
(
    game,
    session,
    track,
    car,
) = "iRacing,1674828783,barcelona gp,Ferrari 488 GT3 Evo 2020".split(",")
all_laps = influx.telemetry_for(game=game, track=track, car=car)

In [3]:
# Resample to 1 meter intervals
df = all_laps[0].copy()
# remove all Gear == 0 rows
df = df[df["Gear"] != 0]
# display(df)
df = analyzer.resample(df, freq=1, columns=["Brake", "SpeedMs", "Throttle", "Gear"])
# # normalise SpeedMs and Gear
df["SpeedMsNormalized"] = df["SpeedMs"] / df["SpeedMs"].max()
df["GearNormalized"] = df["Gear"] / df["Gear"].max()

df

,DistanceRoundTrack,Brake,SpeedMs,Throttle,Gear,id,SpeedMsNormalized,GearNormalized
0,1.164013,0.0,59.151930,1.0,5.0,1674828783-7,0.825622,1.0
1,2.164142,0.0,59.186166,1.0,5.0,1674828783-7,0.826100,1.0
2,3.164272,0.0,59.220282,1.0,5.0,1674828783-7,0.826576,1.0
3,4.164401,0.0,59.254137,1.0,5.0,1674828783-7,0.827048,1.0
4,5.164530,0.0,59.288472,1.0,5.0,1674828783-7,0.827528,1.0
...,...,...,...,...,...,...,...,...
4563,4564.753873,0.0,58.942070,1.0,5.0,1674828783-7,0.822693,1.0
4564,4565.754002,0.0,58.979838,1.0,5.0,1674828783-7,0.823220,1.0
4565,4566.754131,0.0,59.014395,1.0,5.0,1674828783-7,0.823702,1.0
4566,4567.754261,0.0,59.047296,1.0,5.0,1674828783-7,0.824161,1.0


In [4]:
def lap_fig(df):
    fig = go.Figure()

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["SpeedMsNormalized"],
        marker=dict(size=2),
        name="SpeedMsNormalized",
    )

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["Brake"],
        marker=dict(size=1),
        name="Brake",
    )

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["Throttle"],
        marker=dict(size=1),
        name="Throttle",
    )

    fig.add_scatter(
        x=df["DistanceRoundTrack"],
        y=df["GearNormalized"],
        marker=dict(size=1),
        name="GearNormalized",
    )
    return fig


lap_fig(df).show()

In [10]:
# break the track into segements, where throttle input is below 1
#  if you need to brake in a turn, you will release the throttle
#  if you dont need to brake, you might still release the throttle
#  if it's a full throttle turn, you will not release the throttle
#       - we might identify these by looking at the steering angle


def get_change_indices(df, column, threshold=0.9, below=True):
    X = df[column].values
    if below:
        mask = X <= threshold
    else:
        mask = X >= threshold
    change_indices = np.where(np.diff(mask))[0]
    # if len(change_indices) is odd, then the last value is a change
    if len(change_indices) % 2 == 1:
        print("adding last index")
        change_indices = np.append(change_indices, len(X))
    return change_indices


def remove_close_indices(indices, threshold=50):
    # merge throttle changes that are close together
    new_indices = [
        indices[0],
        indices[1],
    ]
    for i in range(2, len(indices), 2):
        start_i = indices[i]
        end_i = indices[i + 1]
        previous_end = indices[i - 1]
        if start_i - previous_end <= threshold:
            new_indices[-1] = end_i
        else:
            new_indices.append(start_i)
            new_indices.append(end_i)
    return new_indices


throttle_changes = get_change_indices(df, "Throttle", threshold=0.95, below=True)
brake_changes = get_change_indices(df, "Brake", threshold=0.005, below=False)
# display(throttle_changes)
# display(brake_changes)

throttle_changes = remove_close_indices(throttle_changes)
brake_changes = remove_close_indices(brake_changes)
# display(throttle_changes)
# display(brake_changes)


fig = lap_fig(df)

# add a vertical line for each index
for idx in throttle_changes:
    fig.add_vline(
        x=df["DistanceRoundTrack"][idx],
        line_width=1,
        line_dash="dash",
        line_color="green",
    )
for idx in brake_changes:
    fig.add_vline(
        x=df["DistanceRoundTrack"][idx],
        line_width=1,
        line_dash="dash",
        line_color="red",
    )

fig.show()

In [13]:
def get_average(df, start_i, end_i, column="Brake", max=True):
    search_df = df[start_i:end_i]

    if max:
        high = abs(round(search_df[column].max(), 2))
        low = high * 0.9
        start = search_df[search_df[column] > low].index.min()
        end = search_df[search_df[column] > low].index.max()
        average = search_df[search_df[column] > low][column].mean()
    else:
        low = abs(round(search_df[column].min(), 2))
        high = low * 1.1
        if low <= 0.1:
            high = 0.1
        start = search_df[search_df[column] < high].index.min()
        end = search_df[search_df[column] < high].index.max()
        average = search_df[search_df[column] < high][column].mean()

    if np.isnan(average):
        raise Exception("average is NaN")

    return {
        "high": high,
        "low": low,
        "avg_start": start,
        "avg_end": end,
        "average": round(average, 2),
    }


fig = lap_fig(df)
segments = []
for i in range(0, len(throttle_changes), 2):
    start_i = throttle_changes[i]
    end_i = throttle_changes[i + 1]
    # find the closest number in the brake_idx array

    brake_i = brake_changes[np.abs(brake_changes - start_i).argmin()]
    # how far is the brake_idx from the start_idx
    distance = np.abs(brake_i - start_i)
    segment = {
        "type": "brake",
        "seg_start": start_i,
        "seg_end": end_i,
        "color": "yellow",
    }
    if distance > 20:
        segment["type"] = "throttle"
        segment["color"] = "green"
        # we are not braking in this segment
        segment["start"] = start_i
        segment["end"] = end_i

        avg_data = get_average(df, start_i, end_i, column="Throttle", max=False)
        segment |= avg_data
    else:
        # search back 20 meters to find the start of the brake
        search_start = brake_i - 20
        if search_start < 0:
            search_start = 0
        search_df = df[search_start:brake_i]
        min = search_df["Brake"].min()
        brake_start_i = search_df[search_df["Brake"] == min].index.max()
        segment["start"] = brake_start_i

        search_df = df[brake_start_i:end_i]
        brake_end_i = search_df[search_df["Brake"] > min].index.max()
        segment["end"] = brake_end_i

        # find the max brake value
        avg_data = get_average(df, brake_i, end_i)
        segment |= avg_data

    fig.add_shape(
        type="rect",
        xref="x",
        yref="y",
        x0=df["DistanceRoundTrack"][avg_data["avg_start"]],
        y0=avg_data["low"],
        x1=df["DistanceRoundTrack"][avg_data["avg_end"]],
        y1=avg_data["high"],
        line=dict(color="orange", width=2),
    )

    fig.add_shape(
        type="line",
        xref="x",
        yref="y",
        x0=df["DistanceRoundTrack"][avg_data["avg_start"]],
        y0=avg_data["average"],
        x1=df["DistanceRoundTrack"][avg_data["avg_end"]],
        y1=avg_data["average"],
        line=dict(color="red", width=2),
    )

    fig.add_shape(
        type="rect",
        xref="x",
        yref="paper",
        x0=df["DistanceRoundTrack"][segment["seg_start"]],
        y0=0,
        x1=df["DistanceRoundTrack"][segment["seg_end"]],
        y1=1,
        line=dict(color=segment["color"], width=2, dash="dot"),
    )
    # get lowest gear in this segment
    segment["gear"] = df["Gear"][segment["seg_start"] : segment["seg_end"]].min()
    segment["speed"] = df["SpeedMs"][segment["seg_start"] : segment["seg_end"]].min()
    segments.append(segment)
fig.show()
pd.DataFrame(segments)

,type,seg_start,seg_end,color,start,end,high,low,avg_start,avg_end,average,gear,speed
0,brake,696,969,yellow,698,862,0.85,0.765,725,777,0.82,2.0,32.179093
1,throttle,1060,1248,green,1060,1248,0.10,0.000,1080,1127,0.00,3.0,41.855462
2,brake,1588,1776,yellow,1588,1709,0.75,0.675,1599,1616,0.73,2.0,29.778647
3,brake,2008,2169,yellow,2007,2099,0.82,0.738,2025,2050,0.80,2.0,24.141404
4,brake,2436,2571,yellow,2434,2513,0.76,0.684,2449,2464,0.73,3.0,33.836430
5,brake,2782,2942,yellow,2784,2842,0.11,0.099,2833,2836,0.11,3.0,44.024236
6,brake,3324,3493,yellow,3324,3451,0.86,0.774,3336,3384,0.84,1.0,17.687205
7,throttle,3572,3805,green,3572,3805,0.10,0.000,3629,3712,0.00,2.0,26.788898
8,brake,3885,4152,yellow,3883,4083,0.87,0.783,4038,4040,0.84,1.0,21.710576
